In [11]:
import itertools
import pickle
import decimal

from pprint import pprint
from datetime import date, datetime, timedelta
from dateutil.relativedelta import relativedelta

daily, partial_month, monthly = 'GA_DIARIO', 'GA_MENSUALPARCIAL', 'GA_MENSUAL'
daily_pfx, partial_month_pfx, monthly_pfx =  daily,  partial_month,  monthly
sufixes = ('', '_sinPlayer', '_xCanal', '_xDispositivo', '_xFuente', '_xHostname', 
           '_xPagina', '_xPais', '_xPais_xDispositivo', '_xPlayer', '_xSocialMedia')

main_tables_gen = (daily, partial_month, monthly)
all_tables_gen = map(lambda t: t[0] + t[1], itertools.product((daily_pfx, monthly_pfx, partial_month_pfx), sufixes))

In [12]:
class Container():
	tables_dir = 'table_picklefiles'

	daily_pfx = 'GA_DIARIO'
	partial_month_pfx = 'GA_MENSUALPARCIAL'
	monthly_pfx = 'GA_MENSUAL'
	
	prefixes = (daily_pfx, partial_month_pfx, monthly_pfx, )
	sufixes = ('', '_sinPlayer', '_xCanal', '_xDispositivo', '_xFuente', '_xHostname', '_xPagina', '_xPais', '_xPais_xDispositivo', '_xPlayer', '_xSocialMedia')

	def __init__(self, table_name):
		if table_name not in map(lambda t: t[0] + t[1], itertools.product(self.prefixes, self.sufixes)):
			raise ValueError(f'{table_name} not a known table')
		
		self.table_name = table_name
		self._load_raw_table(table_name)

		self.column_names = tuple(t[0] for t in self.raw_table[0].cursor_description)

		self._rt_descriptor_dict = {
			col_name : {
				'type' : rem[0],
				'internal_size' : rem[2],
				'precision' : rem[3],
				'scale' : rem[4]
			} for (col_name, *rem) in self.raw_table[0].cursor_description
		}

		self._rt_descriptor_list = [
			{
				'name' : col_desc[0],
				'type' : col_desc[1],
				'internal_size' : col_desc[3],
				'precision' : col_desc[4],
				'scale' : col_desc[5],
				'nullable' : col_desc[6]
			} for col_desc in self.raw_table[0].cursor_description
		]
		self._curate_table()

	def _curate_table(self):
		self.curated_table = []

		for row in self.raw_table:
			row_dict = {}
			for i, field in enumerate(row):
				# row_dict[self._rt_descriptor_list[i]['name']] = float(field) if isinstance(field, decimal.Decimal) else field
				row_dict[self._rt_descriptor_list[i]['name']] = field
			self.curated_table.append(row_dict) 

	def _load_raw_table(self, table_name):
		with open(self.tables_dir + '/' + 'table_' + table_name + '.pkl', 'rb') as pkl_file:
			self.raw_table = pickle.load(pkl_file)
	
	def __getitem__(self, i):
		return self.curated_table[i]
	
	def __str__(self):
		return f'Table(table_name = \'{self.table_name}\', len = {self.__len__()})'
	
	def __len__(self):
		return len(self.curated_table)

In [13]:
class TableIndexError(IndexError):
    pass

# Row fields:
# UKEY | Users | Sessions | Pageviews | PromedioSesión | PorcentajeRebote | Origen | FechaFiltro | FechaCreacion | FechaModificacion
class Row():
    def __init__(self, row):
    
        self.as_dict = {
            'users': row['Users'],
            'sessions': row['Sessions'],
            'pageviews': row['Pageviews'],
            'promediosesion': row['PromedioSesión'],
            'porcentajerebote': row['PorcentajeRebote'],
            'origen': row['Origen'],
            'fechafiltro': datetime.fromisoformat(row['FechaFiltro'])
        }

    def __getitem__(self, s):
        return self.as_dict[s]

    def __repr__(self):
        return self.as_dict.__repr__()

    def __str__(self):
        return self.as_dict.__str__()

    def  __getattr__(self, name):
        return self.as_dict[name]
        
class Table():
    tbl_idx_err = TableIndexError

    def __init__(self, cont):
        self.table = []
        self.table.extend((Row(r) for r in cont))
    
    def __len__(self):
        return len(self.table)
    
    def __getitem__(self, i):
            if 0 <= (i if i >= 0 else (len(self.table) - i)) < len(self.table):
                return self.table[i]
            else:
                raise Table.tbl_idx_err('row position out of range')

daily_tbl, partial_month_tbl, monthly_tbl = tuple(Table(Container(tbl)) for tbl in main_tables_gen)

print(f'len(daily_tbl) = {len(daily_tbl)}')
print(f'len(partial_month_tbl) = {len(partial_month_tbl)}')
print(f'len(monthly_tbl) = {len(monthly_tbl)}')

daily_tbl[len(daily_tbl) - 1]


len(daily_tbl) = 16603
len(partial_month_tbl) = 22194
len(monthly_tbl) = 1256


{'users': Decimal('425799.0000000000'), 'sessions': Decimal('486581.0000000000'), 'pageviews': Decimal('629292.0000000000'), 'promediosesion': '71.0007357459498', 'porcentajerebote': '87.75044648270278', 'origen': 'ViaPais', 'fechafiltro': datetime.datetime(2023, 5, 16, 0, 0)}

In [16]:
daily_origins = set(row['origen'] for row in daily_tbl)
partial_month_origins = set(row['origen'] for row in partial_month_tbl)
monthly_origins = set(row['origen'] for row in monthly_tbl)

origins = daily_origins | partial_month_origins | monthly_origins

print(f'daily_origins         :: {sorted(list(daily_origins))}')
print(f'partial_month_origins :: {sorted(list(partial_month_origins))}')
print(f'monthly_origins       :: {sorted(list(monthly_origins))}')
print(f'origins               :: {sorted(list(origins))}')

daily_by_origin = {}
partial_month_by_origin = {}
monthly_by_origin = {}

for origin in sorted(origins):
    daily_by_origin[origin] = [row for row in daily_tbl if row['origen'] == origin and row['fechafiltro']]
    partial_month_by_origin[origin] = [row for row in partial_month_tbl if row['origen'] == origin and row['fechafiltro']]
    monthly_by_origin[origin] = [row for row in monthly_tbl if row['origen'] == origin and row['fechafiltro']]

    daily_by_origin[origin].sort(key=lambda row : row['fechafiltro'])
    partial_month_by_origin[origin].sort(key=lambda row : row['fechafiltro'])
    monthly_by_origin[origin].sort(key=lambda row : row['fechafiltro'])

for lbl, d in (('daily_by_origin' , daily_by_origin), ('partial_month_by_origin' , partial_month_by_origin), ('monthly_by_origin' , monthly_by_origin)):
    print(lbl)
    for k, v in d.items():
        print(f'\t{k} :: {len(v)}')


daily_origins         :: ['CienRadios', 'Cienradios', 'Ciudad', 'Clarin', 'ElDoce', 'ElTrece', 'La100', 'LaVoz', 'LosAndes', 'Mitre', 'Ole', 'RED CienRadios', 'TN', 'TyC', 'ViaPais']
partial_month_origins :: ['CienRadios', 'Cienradios', 'Ciudad', 'Clarin', 'ElDoce', 'ElTrece', 'Error Mappeo', 'La100', 'LaVoz', 'LosAndes', 'Mitre', 'Ole', 'RED CienRadios', 'TN', 'TyC', 'ViaPais']
monthly_origins       :: ['CienRadios', 'Cienradios', 'Ciudad', 'Clarin', 'ElDoce', 'ElTrece', 'La100', 'LaVoz', 'LosAndes', 'Mitre', 'Ole', 'RED CienRadios', 'TN', 'TyC', 'ViaPais']
origins               :: ['CienRadios', 'Cienradios', 'Ciudad', 'Clarin', 'ElDoce', 'ElTrece', 'Error Mappeo', 'La100', 'LaVoz', 'LosAndes', 'Mitre', 'Ole', 'RED CienRadios', 'TN', 'TyC', 'ViaPais']
daily_by_origin
	CienRadios :: 1236
	Cienradios :: 345
	Ciudad :: 1029
	Clarin :: 1029
	ElDoce :: 1029
	ElTrece :: 1029
	Error Mappeo :: 0
	La100 :: 1581
	LaVoz :: 1018
	LosAndes :: 1029
	Mitre :: 1581
	Ole :: 1029
	RED CienRadios :: 15

In [19]:
(daily_cienradios, partial_month_cienradios, monthly_cienradios) = (
    daily_by_origin['CienRadios'] + daily_by_origin['Cienradios'] + daily_by_origin['RED CienRadios'],                          # daily_cienradios
    partial_month_by_origin['CienRadios'] + partial_month_by_origin['Cienradios'] + partial_month_by_origin['RED CienRadios'],  # partial_month_cienradios
    monthly_by_origin['CienRadios'] + monthly_by_origin['Cienradios'] + monthly_by_origin['RED CienRadios']                     # monthly_cienradios
)

daily_cienradios.sort(key=lambda l : l['fechafiltro'])
partial_month_cienradios.sort(key=lambda l : l['fechafiltro'])
monthly_cienradios.sort(key=lambda l : l['fechafiltro'])

In [20]:
# Check different origins corresponding to 'RED CienRadios' -- Only 'RED CienRadios' is relevant
print('monthly_by_origin')
print('\tCienRadios', monthly_by_origin['CienRadios'][0]['fechafiltro'], '---', monthly_by_origin['CienRadios'][-1]['fechafiltro'], len(monthly_by_origin['CienRadios']))
print('\tCienradios', monthly_by_origin['Cienradios'][0]['fechafiltro'], '---', monthly_by_origin['Cienradios'][-1]['fechafiltro'], len(monthly_by_origin['Cienradios']))
print('\tRED CienRadios', monthly_by_origin['RED CienRadios'][0]['fechafiltro'], '---', monthly_by_origin['RED CienRadios'][-1]['fechafiltro'], len(monthly_by_origin['RED CienRadios']))
print('partial_month_by_origin')
print('\tCienRadios', partial_month_by_origin['CienRadios'][0]['fechafiltro'], '---', partial_month_by_origin['CienRadios'][-1]['fechafiltro'], len(partial_month_by_origin['CienRadios']))
print('\tCienradios', partial_month_by_origin['Cienradios'][0]['fechafiltro'], '---', partial_month_by_origin['Cienradios'][-1]['fechafiltro'], len(partial_month_by_origin['Cienradios']))
print('\tRED CienRadios', partial_month_by_origin['RED CienRadios'][0]['fechafiltro'], '---', partial_month_by_origin['RED CienRadios'][-1]['fechafiltro'], len(partial_month_by_origin['RED CienRadios']))
print('daily_by_origin')
print('\tCienRadios', daily_by_origin['CienRadios'][0]['fechafiltro'], '---', daily_by_origin['CienRadios'][-1]['fechafiltro'], len(daily_by_origin['CienRadios']))
print('\tCienradios', daily_by_origin['Cienradios'][0]['fechafiltro'], '---', daily_by_origin['Cienradios'][-1]['fechafiltro'], len(daily_by_origin['Cienradios']))
print('\tRED CienRadios', daily_by_origin['RED CienRadios'][0]['fechafiltro'], '---', daily_by_origin['RED CienRadios'][-1]['fechafiltro'], len(daily_by_origin['RED CienRadios']))

monthly_by_origin
	CienRadios 2013-01-31 00:00:00 --- 2022-05-31 00:00:00 113
	Cienradios 2022-06-30 00:00:00 --- 2023-04-30 00:00:00 11
	RED CienRadios 2013-01-31 00:00:00 --- 2023-04-30 00:00:00 124
partial_month_by_origin
	CienRadios 2019-01-01 00:00:00 --- 2022-07-30 00:00:00 1238
	Cienradios 2022-06-09 00:00:00 --- 2023-05-16 00:00:00 325
	RED CienRadios 2019-01-01 00:00:00 --- 2023-05-16 00:00:00 1564
daily_by_origin
	CienRadios 2019-01-01 00:00:00 --- 2022-06-05 00:00:00 1236
	Cienradios 2022-06-06 00:00:00 --- 2023-05-16 00:00:00 345
	RED CienRadios 2019-01-01 00:00:00 --- 2023-05-16 00:00:00 1581


In [21]:
for origin in origins:
    print(origin)
    if len(daily_by_origin[origin]) > 0:
        print('\tdaily         : ', daily_by_origin[origin][0]['fechafiltro'], '---', daily_by_origin[origin][-1]['fechafiltro'], len(daily_by_origin[origin]))
    if len(partial_month_by_origin[origin]) > 0:
        print('\tpartial_month : ', partial_month_by_origin[origin][0]['fechafiltro'], '---', partial_month_by_origin[origin][-1]['fechafiltro'], len(partial_month_by_origin[origin]))
    if len(monthly_by_origin[origin]) > 0:
        print('\tmonthly       : ', monthly_by_origin[origin][0]['fechafiltro'], '---', monthly_by_origin[origin][-1]['fechafiltro'], len(monthly_by_origin[origin]))

ViaPais
	daily         :  2020-07-11 00:00:00 --- 2023-05-16 00:00:00 1029
	partial_month :  2019-01-01 00:00:00 --- 2023-05-16 00:00:00 1593
	monthly       :  2017-01-31 00:00:00 --- 2023-04-30 00:00:00 76
Cienradios
	daily         :  2022-06-06 00:00:00 --- 2023-05-16 00:00:00 345
	partial_month :  2022-06-09 00:00:00 --- 2023-05-16 00:00:00 325
	monthly       :  2022-06-30 00:00:00 --- 2023-04-30 00:00:00 11
ElTrece
	daily         :  2020-07-11 00:00:00 --- 2023-05-16 00:00:00 1029
	partial_month :  2019-01-01 00:00:00 --- 2023-05-16 00:00:00 1593
	monthly       :  2017-01-31 00:00:00 --- 2023-04-30 00:00:00 76
Ole
	daily         :  2020-07-11 00:00:00 --- 2023-05-16 00:00:00 1029
	partial_month :  2019-01-01 00:00:00 --- 2023-05-16 00:00:00 1592
	monthly       :  2017-01-31 00:00:00 --- 2023-04-30 00:00:00 76
Mitre
	daily         :  2019-01-01 00:00:00 --- 2023-05-16 00:00:00 1581
	partial_month :  2019-01-01 00:00:00 --- 2023-05-16 00:00:00 1564
	monthly       :  2013-01-31 00:00:

In [22]:
# partial_month_by_origin:

[row for row in partial_month_by_origin['Error Mappeo']]
[row['fechafiltro'] for row in partial_month_by_origin['Error Mappeo']]

[datetime.datetime(2022, 6, 17, 0, 0),
 datetime.datetime(2022, 12, 20, 0, 0),
 datetime.datetime(2022, 12, 21, 0, 0),
 datetime.datetime(2022, 12, 22, 0, 0),
 datetime.datetime(2022, 12, 23, 0, 0),
 datetime.datetime(2022, 12, 24, 0, 0),
 datetime.datetime(2022, 12, 25, 0, 0),
 datetime.datetime(2022, 12, 26, 0, 0),
 datetime.datetime(2022, 12, 27, 0, 0),
 datetime.datetime(2022, 12, 28, 0, 0)]

In [ ]:
def historic(origin, partial, monthly, day_date):
    partial_for_date = [row for row in partial[origin] if date.fromisoformat(row['FechaFiltro']) == day_date]
    
    month_date = day_date.replace(day=1) #  + relativedelta.relativedelta(month=1) # - relativedelta.relativedelta(day=1)
    month_date = month_date + relativedelta(months=+1, days=-1)
    monthly_for_date = [row for row in monthly[origin] if date.fromisoformat(row['FechaFiltro']) == month_date]

    if len(partial_for_date) == 1 and len(monthly_for_date) == 1:
        return partial_for_date[0], monthly_for_date[0]
    else:
        return None, None

def ratio(origin, day):
    partial_for_day, monthly_for_day = historic(origin, partial_month_by_origin, monthly_by_origin, day)

    if partial_for_day and monthly_for_day:
        return partial_for_day['Users']/monthly_for_day['Users']
    else:
        # print('partial_for_day :', partial_for_day['Users'] if partial_for_day else None)
        # print('monthly_for_day :', monthly_for_day['Users'] if monthly_for_day else None)
        return None

def predict(origin, day):
    rya = ratio(origin, day - relativedelta(years=1)) # ratio-year-ago

    if rya:
        if (partial_for_day := [row for row in partial_month_by_origin[origin] if date.fromisoformat(row['FechaFiltro']) == day]):
            if (partial_row := partial_for_day[0]):
                partial = partial_row['Users']
                return partial/rya
        else:
            return None
    else:
        return None

def predict2(origin, day, weight):
    rya = ratio(origin, day - relativedelta(years=1)) # ratio-year-ago
    r2ya = ratio(origin, day - relativedelta(years=2)) # ratio-2year-ago

    if rya and r2ya:
        if (partial_for_day := [row for row in partial_month_by_origin[origin] if date.fromisoformat(row['FechaFiltro']) == day]):
            if (partial_row := partial_for_day[0]):
                partial = partial_row['Users']
                return decimal.Decimal(weight) * partial/rya + (1 - decimal.Decimal(weight)) * partial/r2ya
                # return (weight) * partial/rya + (1 - (weight)) * partial/r2ya
        else:
            return None
    else:
        return None


In [ ]:
for i, day in enumerate(date.fromisoformat('2023-01-01') + relativedelta(days=i) for i in range(0, 120)):
    prediction = predict('RED CienRadios', day)
    end_of_month = day.replace(day=1) + relativedelta(months=+1, days=-1)
    day_past_year = day + relativedelta(years=-1)
    
    partial_of_day = [row for row in partial_month_by_origin['RED CienRadios'] if date.fromisoformat(row['FechaFiltro']) == day][0]['Users']

    actual = rows[0]['Users'] if (rows := [row for row in monthly_by_origin['RED CienRadios'] if date.fromisoformat(row['FechaFiltro']) == end_of_month]) else None
    if not actual or not prediction: 
        print(f'{"actual is None" if not actual else ""}{"prediction is None" if not prediction else ""}')
        continue
    print(f'{day} Predicted : {prediction:>12.12} | Real ({end_of_month}): {actual:>12.12} | ratio : {ratio("RED CienRadios", day_past_year):.3} | partial_of_day : {partial_of_day} | abs_err : {prediction-actual:>+12.12} | rel_err : {(prediction-actual)/actual:>+12.12} | rel_err(%) : {(prediction-actual)/actual * 100:>+12.12}')
    # print(f'{day} --------- : {partial_of_day/ratio("RED CienRadios", day_past_year):>12.12} |')

2023-01-01 Predicted : 42852619.7932 | Real (2023-01-31): 28696062.0000 | ratio : 0.0462 | partial_of_day : 1980943.0000000000 | abs_err : +14156557.7932 | rel_err : +0.493327544148 | rel_err(%) : +49.3327544148
2023-01-02 Predicted : 44818116.1666 | Real (2023-01-31): 28696062.0000 | ratio : 0.0813 | partial_of_day : 3644533.0000000000 | abs_err : +16122054.1666 | rel_err : +0.561821136525 | rel_err(%) : +56.1821136525
2023-01-03 Predicted : 35238851.8972 | Real (2023-01-31): 28696062.0000 | ratio : 0.147 | partial_of_day : 5181961.0000000000 | abs_err : +6542789.89716 | rel_err : +0.228003058300 | rel_err(%) : +22.8003058300
2023-01-04 Predicted : 31338756.1817 | Real (2023-01-31): 28696062.0000 | ratio : 0.202 | partial_of_day : 6342104.0000000000 | abs_err : +2642694.18166 | rel_err : +0.0920925728992 | rel_err(%) : +9.20925728992
2023-01-05 Predicted : 29438373.4874 | Real (2023-01-31): 28696062.0000 | ratio : 0.256 | partial_of_day : 7541584.0000000000 | abs_err : +742311.487421 

In [ ]:
for i, day in enumerate(date.fromisoformat(f'2023-01-01') + relativedelta(days=i) for i in range(0, 120)):
    prediction = predict2('RED CienRadios', day, 0.24)
    end_of_month = day.replace(day=1) + relativedelta(months=+1, days=-1)

    actual = rows[0]['Users'] if (rows := [row for row in monthly_by_origin['RED CienRadios'] if date.fromisoformat(row['FechaFiltro']) == end_of_month]) else None
    if not actual or not prediction: 
        print(f'{"actual is None" if not actual else ""}{"prediction is None" if not prediction else ""}')
        continue
    print(f'{day} Predicted : {prediction:>12.12} | Real ({end_of_month}): {actual:>12.12} | abs_err : {prediction-actual:>+12.12} | rel_err : {(prediction-actual)/actual:>+12.12} | rel_err(%) : {(prediction-actual)/actual * 100:>+12.12}')

2023-01-01 Predicted : 39371721.1536 | Real (2023-01-31): 28696062.0000 | abs_err : +10675659.1536 | rel_err : +0.372025233064 | rel_err(%) : +37.2025233064
2023-01-02 Predicted : 37022320.0597 | Real (2023-01-31): 28696062.0000 | abs_err : +8326258.05965 | rel_err : +0.290153333919 | rel_err(%) : +29.0153333919
2023-01-03 Predicted : 35673179.3449 | Real (2023-01-31): 28696062.0000 | abs_err : +6977117.34486 | rel_err : +0.243138495619 | rel_err(%) : +24.3138495619
2023-01-04 Predicted : 33575882.5214 | Real (2023-01-31): 28696062.0000 | abs_err : +4879820.52135 | rel_err : +0.170051922851 | rel_err(%) : +17.0051922851
2023-01-05 Predicted : 30478919.8192 | Real (2023-01-31): 28696062.0000 | abs_err : +1782857.81924 | rel_err : +0.0621290063855 | rel_err(%) : +6.21290063855
2023-01-06 Predicted : 28683016.5571 | Real (2023-01-31): 28696062.0000 | abs_err : -13045.4429243 | rel_err : -0.000454607427467 | rel_err(%) : -0.0454607427467
2023-01-07 Predicted : 28797932.5660 | Real (2023-01

In [ ]:
acc_list = []
for lbl in range(0, 1000):
    acc = 0
    for i, day in enumerate(date.fromisoformat(f'2022-01-01') + relativedelta(days=i) for i in range(0, 365)):
        actual = rows[0]['Users'] if (rows := [row for row in monthly_by_origin['RED CienRadios'] if date.fromisoformat(row['FechaFiltro']) == end_of_month]) else None
        monthly_by_origin['RED CienRadios']
        predict = predict2('RED CienRadios', day, lbl/1000)

        if actual == None or predict == None:
            continue

        acc += (actual - predict) ** 2
    
    acc_list.append((lbl, acc))
    print(f'For w = {lbl:3} :: {acc}')



For w =   0 :: 23037131587605786.00137753726
For w =   1 :: 23035225498193004.10626166621
For w =   2 :: 23033327214169519.46237186326
For w =   3 :: 23031436735535332.06970812851
For w =   4 :: 23029554062290441.92827046170
For w =   5 :: 23027679194434849.03805886327
For w =   6 :: 23025812131968553.39907333287
For w =   7 :: 23023952874891555.01131387055
For w =   8 :: 23022101423203853.87478047651
For w =   9 :: 23020257776905449.99106887381
For w =  10 :: 23018421935996343.35539189270
For w =  11 :: 23016593900476533.97411888609
For w =  12 :: 23014773670346021.84090758155
For w =  13 :: 23012961245604806.96207317085
For w =  14 :: 23011156626252889.33132754269
For w =  15 :: 23009359812290268.95493172831
For w =  16 :: 23007570803716945.82665177668
For w =  17 :: 23005789600532919.94961143343
For w =  18 :: 23004016202738191.32994986781
For w =  19 :: 23002250610332759.95536166070
For w =  20 :: 23000492823316625.83201306220
For w =  21 :: 22998742841689788.95990407176
For w =  2

In [ ]:
def days_until_end_of_month(day):
    return ((day.replace(day=1) + relativedelta(months=+1, days=-1)) - day).days

def intermonth(day):
    return day.replace(day=1) + relativedelta(days=-1) - relativedelta(days=days_until_end_of_month(day))

print(intermonth(date.fromisoformat('2022-03-02')))

2022-01-30


* Cuantos dias faltan para llegar al ultimo dia del mes? -- busco esos mismos dias del mes pasado



In [ ]:
acc_list_sorted = sorted(acc_list, key=lambda l: l[1])
acc_list_int = [(w, int(s)) for (w, s) in acc_list]
avg = min(s for (w,s) in acc_list_int)
dev_list = [(w, s - avg) for (w, s) in acc_list_int]
dev_list


[(0, 233688820036726),
 (1, 231782730623944),
 (2, 229884446600459),
 (3, 227993967966272),
 (4, 226111294721381),
 (5, 224236426865789),
 (6, 222369364399493),
 (7, 220510107322495),
 (8, 218658655634793),
 (9, 216815009336389),
 (10, 214979168427283),
 (11, 213151132907473),
 (12, 211330902776961),
 (13, 209518478035746),
 (14, 207713858683829),
 (15, 205917044721208),
 (16, 204128036147885),
 (17, 202346832963859),
 (18, 200573435169131),
 (19, 198807842763699),
 (20, 197050055747565),
 (21, 195300074120728),
 (22, 193557897883189),
 (23, 191823527034946),
 (24, 190096961576001),
 (25, 188378201506353),
 (26, 186667246826003),
 (27, 184964097534950),
 (28, 183268753633193),
 (29, 181581215120735),
 (30, 179901481997573),
 (31, 178229554263709),
 (32, 176565431919141),
 (33, 174909114963872),
 (34, 173260603397899),
 (35, 171619897221224),
 (36, 169986996433846),
 (37, 168361901035765),
 (38, 166744611026981),
 (39, 165135126407495),
 (40, 163533447177306),
 (41, 161939573336414),
 (

In [ ]:
from matplotlib import pyplot

for i, day in enumerate(date.fromisoformat('2023-01-01') + relativedelta(days=i) for i in range(0, 120)):
    partial_of_day = [row for row in partial_month_by_origin['RED CienRadios'] if date.fromisoformat(row['FechaFiltro']) == day]
    print(day, partial_of_day[0])

# pyplot.scatter()

2023-01-01 {'UKEY': '2023-01-01RED CienRadios', 'Users': Decimal('1980943.0000000000'), 'Sessions': Decimal('2466549.0000000000'), 'Pageviews': Decimal('3350986.0000000000'), 'PromedioSesión': '122.86432055475079', 'PorcentajeRebote': '56.513047176439635', 'Origen': 'RED CienRadios', 'FechaFiltro': '2023-01-01', 'FechaCreacion': datetime.datetime(2023, 1, 11, 7, 29, 43), 'FechaModificacion': None}
2023-01-02 {'UKEY': '2023-01-02RED CienRadios', 'Users': Decimal('3644533.0000000000'), 'Sessions': Decimal('5109832.0000000000'), 'Pageviews': Decimal('7272523.0000000000'), 'PromedioSesión': '151.74088228340972', 'PorcentajeRebote': '53.06012017616235', 'Origen': 'RED CienRadios', 'FechaFiltro': '2023-01-02', 'FechaCreacion': datetime.datetime(2023, 1, 12, 7, 24, 59), 'FechaModificacion': None}
2023-01-03 {'UKEY': '2023-01-03RED CienRadios', 'Users': Decimal('5181961.0000000000'), 'Sessions': Decimal('8108790.0000000000'), 'Pageviews': Decimal('11796837.0000000000'), 'PromedioSesión': '163.

In [1]:
# Variables Adicionales a Considerar:
#   - Dia correspondiente del mes previo, contando desde atras
#   - Dia de la semana {0 .. 6}
#   - Si dia laboral (True/False)
#
#   - Medicion diaria : sirve entender si voy creciendo o bajando -- contemplar cambios del algoritmo de Google. 
#   - Variacion diaria!!! ... rangos? 
#       - Volatilidad segun impacto de notas

#   - Cambios bruscos
#   - Tendencia a acumular usuarios unicos / Grandes variaciones == Alarmantes
